# OA Attributes Normalised
Extract header attributes from OA documents using LLM (GPT-4).

**Input**: `bronze_md.md_oa_file_raw`, `silver_md.md_dta_operational_agreement_draft`

**Output**: `silver_md.md_dta_oa_attributes_draft`

Extracts: trial_protocol, data_provider, data_recipient, type_of_data, document_version, issue_date


In [ ]:
# Cell 0: Imports
import json
import requests
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType
from clinical_data_standards_framework.utils import save_with_audit, compute_definition_hash

print("✅ Framework loaded successfully")


In [ ]:
# Cell 1: Read config & globals from setup task

globals_dict = json.loads(
    dbutils.jobs.taskValues.get(taskKey="setup", key="globals")
)
services_dict = json.loads(
    dbutils.jobs.taskValues.get(taskKey="setup", key="services")
)

try:
    pipeline_config = json.loads(
        dbutils.jobs.taskValues.get(taskKey="setup", key="pipeline_config")
    )
except Exception:
    pipeline_config = None

flow_name = dbutils.jobs.taskValues.get(taskKey="setup", key="flow_name")

if not pipeline_config:
    raise ValueError(
        "pipeline_config task value is missing; "
        "ensure setup task ran with flow_name='operational_agreement_processor'"
    )

# Audit globals
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id    = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name  = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id    = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

# From globals
catalog       = globals_dict["catalog"]
bronze_schema = globals_dict["bronze_schema"]
silver_schema = globals_dict["silver_schema"]

print(f"Flow from setup: {flow_name}")
print(f"Catalog: {catalog}")
print(f"Bronze schema: {bronze_schema}")
print(f"Silver schema: {silver_schema}")


In [ ]:
# Cell 2: Resolve OA pipeline configuration and define tables

docs_cfg   = pipeline_config["documents"]["operational_agreement"]
output_cfg = docs_cfg["output"]

raw_table_name = output_cfg["raw_table_name"]  # e.g. "md_oa_file_raw"

raw_table_full      = f"{catalog}.{bronze_schema}.{raw_table_name}"
oa_entities_table   = f"{catalog}.{silver_schema}.md_dta_operational_agreement_draft"
oa_attributes_table = f"{catalog}.{silver_schema}.md_dta_oa_attributes_draft"

print("\nResolved OA tables:")
print(f"  raw_table_full        = {raw_table_full}")
print(f"  oa_entities_table     = {oa_entities_table}")
print(f"  oa_attributes_table   = {oa_attributes_table}")

spark.sql(f"USE CATALOG `{catalog}`")

# Definition hash helper
@F.udf(returnType=StringType())
def oa_definition_hash_udf(
    operational_agreement_id: str,
    trial_id: str,
    data_stream: str,
    data_provider_name: str
) -> str:
    """
    Compute a stable definition hash for OA header / attribute rows.
    """
    record = {
        "operational_agreement_id": operational_agreement_id or "",
        "trial_id": trial_id or "",
        "data_stream": data_stream or "",
        "data_provider_name": data_provider_name or "",
    }
    return compute_definition_hash(record, list(record.keys()))


In [ ]:
# Cell 3: Read source data and join

# Bronze OA content (one row per document)
oa_raw_df = spark.table(raw_table_full).select(
    "document_id",
    "parent_document_id",
    "extracted_path",
    "file_name",
    "file_extension",
    "file_version",
    "content"
)

print(f"\nRows in raw OA table ({raw_table_full}): {oa_raw_df.count()}")

# Silver OA entities (one row per parent_document_id / OA)
oa_base_df = spark.table(oa_entities_table)
oa_cols = oa_base_df.columns

# Handle possible column name variants defensively
select_cols = [
    "operational_agreement_id",
    "parent_document_id",
    "dta_id",
    "version_tag",
    "version_status",
    "is_current_draft",
]

if "trial_id" in oa_cols:
    select_cols.append("trial_id")

if "data_stream" in oa_cols:
    select_cols.append("data_stream")
elif "data_stream_type" in oa_cols:
    select_cols.append("data_stream_type")

if "data_provider" in oa_cols:
    select_cols.append("data_provider")
elif "data_provider_name" in oa_cols:
    select_cols.append("data_provider_name")

oa_entities_df = oa_base_df.select(*[F.col(c) for c in select_cols])

print(f"Rows in OA entities table ({oa_entities_table}): {oa_entities_df.count()}")

# Join on parent_document_id to bring FK + versioning into this notebook
joined_source_df = (
    oa_raw_df.alias("raw")
    .join(
        oa_entities_df.alias("oa"),
        on="parent_document_id",
        how="left"
    )
)

print(f"\nJoined rows (raw + OA entities): {joined_source_df.count()}")

# If the attributes table already exists, avoid duplicating records by document_id
if spark.catalog.tableExists(oa_attributes_table):
    existing_docs_df = spark.table(oa_attributes_table).select("document_id").distinct()
    joined_source_df = joined_source_df.join(
        existing_docs_df,
        on="document_id",
        how="left_anti"
    )
    print(f"Rows to process after excluding already attributed docs: {joined_source_df.count()}")
else:
    print(f"Attributes table does not yet exist: {oa_attributes_table}")
    print("Processing all joined rows.")

# Early exit if nothing to do
row_count = joined_source_df.count()
if row_count == 0:
    print("\n✓ No new OA documents to attribute. Exiting.")
    dbutils.notebook.exit("No new OA attribute rows to process")


In [ ]:
# Cell 4: Model endpoint + generic prompt for header attributes

url = "https://imuj8rzcm7.execute-api.us-east-1.amazonaws.com/dev/genai-chat?api-version=2023-03-15-preview%22"

api_key = dbutils.secrets.get("dta_poc_api_key", "gpt_api_key")

headers = {
    "api-key": api_key,
    "Content-Type": "application/json",
}

MODEL_NAME = "gpt-4.1-mini-global"

doc_prompt_header = """
You are analysing the text of a trial-specific data transfer agreement (tsDTA)
Operational Agreement document.

The document may contain multiple pages concatenated together and may include
page headers or footers such as "Page X of Y". These headers and footers are
not part of the logical content and must be ignored when extracting entities.

Focus on the introductory header block where fields such as the following
typically appear (labels may vary slightly but are semantically similar):

- Trial Protocol
- Data Provider
- Data Recipient
- Type of Data
- Document Version
- Issue Date

Your task is to identify and extract the following fields, based ONLY on the
text of this document:

1) trial_protocol - The trial protocol identifier or name
2) data_provider - The organization providing the data
3) data_recipient - The organization receiving the data
4) type_of_data - The category or type of data (e.g., Lab, ECG, PK)
5) document_version - The version number or identifier of this document
6) issue_date - The date this document was issued

OUTPUT FORMAT

You MUST return a single JSON object with exactly these fields:

{
  "trial_protocol": string,
  "data_provider": string,
  "data_recipient": string,
  "type_of_data": string,
  "document_version": string,
  "issue_date": string
}

Rules:
- Do NOT include any additional fields.
- Do NOT include any explanatory text before or after the JSON object.
- Use empty strings "" when a value cannot be found.
- Base your answers only on the content provided in this document.

----- DOCUMENT TEXT START -----
""".strip()

doc_prompt_footer = "\n----- DOCUMENT TEXT END -----"

print("✅ Model endpoint and prompt configured")


In [ ]:
# Cell 5: Extraction function and UDF

def extract_header_attributes(doc_text: str) -> str:
    """
    Call the GPT endpoint once per document and return a CLEAN JSON string
    with header attribute fields.
    """
    empty_result = {
        "trial_protocol": "",
        "data_provider": "",
        "data_recipient": "",
        "type_of_data": "",
        "document_version": "",
        "issue_date": ""
    }
    
    if doc_text is None or doc_text.strip() == "":
        return json.dumps(empty_result)

    full_prompt = f"{doc_prompt_header}\n{doc_text}\n{doc_prompt_footer}"

    data = {
        "model": MODEL_NAME,
        "messages": [
            {
                "role": "system",
                "content": "You are a precise information extraction assistant that outputs strict JSON only."
            },
            {
                "role": "user",
                "content": full_prompt,
            },
        ],
    }

    try:
        resp = requests.post(url, headers=headers, json=data, timeout=120)
        resp.raise_for_status()
        resp_json = resp.json()
        raw_content = resp_json["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"⚠ Error calling OA attributes model: {e}")
        return json.dumps(empty_result)

    if not raw_content:
        return json.dumps(empty_result)

    # Clean the model output down to pure JSON
    text = raw_content.strip()
    start = text.find("{")
    end = text.rfind("}")

    if start != -1 and end != -1 and end > start:
        json_candidate = text[start : end + 1]
    else:
        print("⚠ Could not find JSON braces in model output; falling back to empty JSON.")
        return json.dumps(empty_result)

    try:
        parsed = json.loads(json_candidate)
        cleaned = {
            "trial_protocol": parsed.get("trial_protocol", "") or "",
            "data_provider": parsed.get("data_provider", "") or "",
            "data_recipient": parsed.get("data_recipient", "") or "",
            "type_of_data": parsed.get("type_of_data", "") or "",
            "document_version": parsed.get("document_version", "") or "",
            "issue_date": parsed.get("issue_date", "") or "",
        }
        return json.dumps(cleaned)
    except Exception as e:
        print(f"⚠ Failed to parse JSON from model output: {e}")
        return json.dumps(empty_result)


@F.udf(returnType=StringType())
def header_attrs_json_udf(doc_text: str) -> str:
    return extract_header_attributes(doc_text)

print("✅ Header attributes extraction UDF defined")


In [ ]:
# Cell 6: Call the model and parse JSON into columns

# Extract header attributes using LLM
attrs_json_df = joined_source_df.select(
    "document_id",
    "parent_document_id",
    "operational_agreement_id",
    "dta_id",
    "trial_id",
    F.col("data_stream").alias("data_stream") if "data_stream" in joined_source_df.columns else F.col("data_stream_type").alias("data_stream"),
    F.col("data_provider_name").alias("data_provider_name") if "data_provider_name" in joined_source_df.columns else F.col("data_provider").alias("data_provider_name"),
    "file_name",
    "file_extension",
    "version_tag",
    "version_status",
    "is_current_draft",
    "content",
    header_attrs_json_udf(F.col("content")).alias("attrs_json"),
)

# Parse JSON into columns
attrs_schema = StructType([
    StructField("trial_protocol", StringType(), True),
    StructField("data_provider", StringType(), True),
    StructField("data_recipient", StringType(), True),
    StructField("type_of_data", StringType(), True),
    StructField("document_version", StringType(), True),
    StructField("issue_date", StringType(), True),
])

parsed_attrs_df = attrs_json_df.select(
    "*",
    F.from_json("attrs_json", attrs_schema).alias("attrs"),
).select(
    "document_id",
    "parent_document_id",
    "operational_agreement_id",
    "dta_id",
    "trial_id",
    "data_stream",
    "data_provider_name",
    "file_name",
    "file_extension",
    "version_tag",
    "version_status",
    "is_current_draft",
    F.col("attrs.trial_protocol").alias("trial_protocol"),
    F.col("attrs.data_provider").alias("data_provider"),
    F.col("attrs.data_recipient").alias("data_recipient"),
    F.col("attrs.type_of_data").alias("type_of_data"),
    F.col("attrs.document_version").alias("document_version"),
    F.col("attrs.issue_date").alias("issue_date"),
)

print("\nSample of extracted attributes:")
parsed_attrs_df.show(5, truncate=False)


In [ ]:
# Cell 7: Build final dataframe and write to Silver

# Add UUID, definition hash, status, and vendor_comments
final_attrs_df = (
    parsed_attrs_df
    .withColumn("oa_attribute_id", F.expr("uuid()"))
    .withColumn(
        "definition_hash",
        oa_definition_hash_udf(
            F.col("operational_agreement_id"),
            F.col("trial_id"),
            F.col("data_stream"),
            F.col("data_provider_name"),
        )
    )
    .withColumn("status", F.lit("COMPLETED"))
    .withColumn("row_status", F.lit("COMPLETED"))
    .withColumn("vendor_comments", F.lit(None).cast("string"))
)

# Reorder columns for final output
final_attrs_df = final_attrs_df.select(
    "oa_attribute_id",
    "operational_agreement_id",
    "document_id",
    "parent_document_id",
    "dta_id",
    "trial_id",
    "data_stream",
    "data_provider_name",
    "trial_protocol",
    "data_provider",
    "data_recipient",
    "type_of_data",
    "document_version",
    "issue_date",
    "definition_hash",
    "status",
    "row_status",
    "vendor_comments",
    "file_name",
    "file_extension",
    "version_tag",
    "version_status",
    "is_current_draft",
)

attrs_count = final_attrs_df.count()
print(f"\nRows to write to {oa_attributes_table}: {attrs_count}")

if attrs_count == 0:
    dbutils.notebook.exit("No OA attribute rows to write")

# Write to Silver table
save_with_audit(
    df=final_attrs_df,
    table_name=oa_attributes_table,
    created_by_principal=created_by_principal,
    databricks_job_id=databricks_job_id,
    databricks_job_name=databricks_job_name,
    databricks_run_id=databricks_run_id,
    mode="append",
    table_description=(
        "Normalized OA header attributes extracted from operational agreement documents. "
        "Includes trial_protocol, data_provider, data_recipient, type_of_data, document_version, issue_date."
    ),
)

print(f"\n✅ Successfully wrote {attrs_count} row(s) to {oa_attributes_table}")

# Set task values for downstream notebooks
dbutils.jobs.taskValues.set(key="oa_attributes_status", value="SUCCESS")
dbutils.jobs.taskValues.set(key="oa_attributes_count", value=str(attrs_count))
